In [1]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Predicting Movie Review Sentiment with BERT on TF Hub

If you’ve been following Natural Language Processing over the past year, you’ve probably heard of BERT: Bidirectional Encoder Representations from Transformers. It’s a neural network architecture designed by Google researchers that’s totally transformed what’s state-of-the-art for NLP tasks, like text classification, translation, summarization, and question answering.

Now that BERT's been added to [TF Hub](https://www.tensorflow.org/hub) as a loadable module, it's easy(ish) to add into existing Tensorflow text pipelines. In an existing pipeline, BERT can replace text embedding layers like ELMO and GloVE. Alternatively, [finetuning](http://wiki.fast.ai/index.php/Fine_tuning) BERT can provide both an accuracy boost and faster training time in many cases.

Here, we'll train a model to predict whether an IMDB movie review is positive or negative using BERT in Tensorflow with tf hub. Some code was adapted from [this colab notebook](https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb). Let's get started!

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0509 07:55:33.073018 140673591916288 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [3]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [4]:
!mkdir bert_agreement

mkdir: cannot create directory ‘bert_agreement’: File exists


In [5]:
OUTPUT_DIR = "bert_agreement"

In [6]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'bert_agreement'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: bert_agreement *****


# Data

First, let's download the dataset, hosted by Stanford. The code below, which downloads, extracts, and imports the IMDB Large Movie Review Dataset, is borrowed from [this Tensorflow tutorial](https://www.tensorflow.org/hub/tutorials/text_classification_with_tf_hub).

In [7]:
from tensorflow import keras
import os
import re

In [8]:
Conventional = ["Generic correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Choice of tense", "Tense form", "Voice", "Modals", "Verb pattern", "Intransitive verb", "Transitive verb", "Reflexive verb", "Verb with as", "Ambitransitive verb", "Two verbal forms in the predicate", "Verb + Infinitive", "Verb + Gerund", "Verb + Infinitive OR Gerund", "Verb + Bare Infinitive", "Verb + Object/Addressee + Bare Infinitive", "Infinitive Restoration Alternation", "Verb + Participle", "Get + participle", "Complex-object verb", "Verbal idiom", "Prepositional or phrasal verb", "Dative verb with alternation", "Verb followed by a clause", "Verb + that/WH + Clause", "Verb + if/whether + clause", "Verb + that + Subjunctive clause", "Verb + it + Conj + Clause", "Participial construction", "Infinitive construction", "Gerund phrase", "Nouns", "Countable/uncountable", "Prepositional noun", "Possessive form of a noun", "Noun as an attribute", "Noun + Infinitive", "Noun number", "Prepositions", "Conjunctions", "Adjectives", "Comparative degree of adjectives", "Superlative degree of adjectives", "Prepositional adjective", "Adjective as a collective noun", "Adverbs", "Comparative degree of adverbs", "Superlative degree of adverbs", "Prepositional adverb", "Numerals", "Pronouns", "Agreement", "Word order", "Standard word order", "Emphatic shift", "Cleft sentence", "Interrogative word order", "Incomplete sentence", "Exclamation", "Title structure", "Note structure", "Conditionals", "Attributes", "Relative clause", "Defining relative clause", "Non-defining relative clause", "Coordinate relative clause", "Attributive participial construction", "Parallel constructions", "Negation", "Comparative construction", "Numerical comparison", "Confusion of structures", "Vocabulary", "Word choice", "Choice of lexical item", "Words often confused", "Choice of a part of lexical item", "Absence of certain components of a collocation", "Redundant word(s)", "Word formation", "Derivational affixes", "Formational suffix", "Formational prefix", "Confusion of categories", "Compound word", "Discourse", "Referential device", "Coherence", "Linking device", "Inappropriate register", "Absence of a component in clause or sentence", "Redundant component in clause or sentence", "Absence of necessary explanation or detail", "Deletion"]
Tags = ["Correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Tense_choice", "Tense_form", "Voice", "Modals", "Verb_pattern", "Intransitive", "Transitive", "Reflexive_verb", "Presentation", "Ambitransitive", "Two_in_a_row", "Verb_Inf", "Verb_Gerund", "Verb_Inf_Gerund", "Verb_Bare_Inf", "Verb_object_bare", "Restoration_alter", "Verb_part", "Get_part", "Complex_obj", "Verbal_idiom", "Prepositional_verb", "Dative", "Followed_by_a_clause", "that_clause", "if_whether_clause", "that_subj_clause", "it_conj_clause", "Participial_constr", "Infinitive_constr", "Gerund_phrase", "Nouns", "Countable_uncountable", "Prepositional_noun", "Possessive", "Noun_attribute", "Noun_inf", "Noun_number", "Prepositions", "Conjunctions", "Adjectives", "Comparative_adj", "Superlative_adj", "Prepositional_adjective", "Adj_as_collective", "Adverbs", "Comparative_adv", "Superlative_adv", "Prepositional_adv", "Numerals", "Pronouns", "Agreement_errors", "Word_order", "Standard", "Emphatic", "Cleft", "Interrogative", "Abs_comp_clause", "Exclamation", "Title_structure", "Note_structure", "Conditionals", "Attributes", "Relative_clause", "Defining", "Non_defining", "Coordinate", "Attr_participial", "Lack_par_constr", "Negation", "Comparative_constr", "Numerical", "Confusion_of_structures", "Vocabulary", "Word_choice", "lex_item_choice", "Often_confused", "lex_part_choice", "Absence_comp_colloc", "Redundant", "Derivation", "Formational_affixes", "Suffix", "Prefix", "Category_confusion", "Compound_word", "Discourse", "Ref_device", "Coherence", "Linking_device", "Inappropriate_register", "Absence_comp_sent", "Redundant_comp", "Absence_explanation", "delete"]
translate_dict = {e[0]: e[1] for e in zip(Conventional, Tags)}

error_type = "Agreement" #@param ["Spelling", "Choice of lexical item", "Deletion", "Prepositions", "Agreement", "Noun number", "Confusion of categories", "Referential device", "Capitalisation", "Words often confused"]
error_ratio = "ToFifteen" #@param ["ToFifteen", "AugmentedRatio"]

error_type = translate_dict[error_type]

It's about time we named our model

In [9]:
modelname = "Agreement_ToFifteen" # @param {type:"string"}

In [10]:
import shutil

filename = error_ratio+"/train/"+error_type+".json"
shutil.copy2('./Datasets/'+filename,'.')
trn = pd.read_json(error_type+".json").reset_index(drop=True)
trn["is_error"] = trn["is_error"].astype(int)

filename = error_ratio+"/test/"+error_type+".json"
shutil.copy2('./Datasets/'+filename,'./'+error_type+'_test.json')
tst = pd.read_json(error_type+"_test.json").reset_index(drop=True)
tst["is_error"] = tst["is_error"].astype(int)

In [11]:
len(trn)

51264

To keep training fast, we'll take a sample of 5000 train and test examples, respectively.

In [12]:
train = trn
test = tst

In [13]:
train.columns

Index(['context', 'is_error', 'path', 'substring'], dtype='object')

For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [14]:
DATA_COLUMN = 'context'
SUBSTR_COLUMN = 'substring'
LABEL_COLUMN = 'is_error'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [15]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [16]:
train_InputExamples

0        <bert.run_classifier.InputExample object at 0x...
1        <bert.run_classifier.InputExample object at 0x...
2        <bert.run_classifier.InputExample object at 0x...
3        <bert.run_classifier.InputExample object at 0x...
4        <bert.run_classifier.InputExample object at 0x...
5        <bert.run_classifier.InputExample object at 0x...
6        <bert.run_classifier.InputExample object at 0x...
7        <bert.run_classifier.InputExample object at 0x...
8        <bert.run_classifier.InputExample object at 0x...
9        <bert.run_classifier.InputExample object at 0x...
10       <bert.run_classifier.InputExample object at 0x...
11       <bert.run_classifier.InputExample object at 0x...
12       <bert.run_classifier.InputExample object at 0x...
13       <bert.run_classifier.InputExample object at 0x...
14       <bert.run_classifier.InputExample object at 0x...
15       <bert.run_classifier.InputExample object at 0x...
16       <bert.run_classifier.InputExample object at 0x.

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [17]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1"

class NonMaskOmittingTokenizer(bert.tokenization.FullTokenizer):
  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text):
      for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(sub_token)

    for index, item in enumerate(split_tokens):
      if index >= len(split_tokens)-2:
        break
      if item == '[' and split_tokens[index + 1] == 'MA' and split_tokens[index + 2] == '##S' and split_tokens[index + 3] == '##K' and split_tokens[index + 4] == ']':
        split_tokens[index] = "[MASK]"
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]

    return split_tokens

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return NonMaskOmittingTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0509 07:56:03.605319 140673591916288 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [18]:
tokenizer.tokenize("This here's an example of using the BERT [MASK] tokenizer")

['This',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'B',
 '##ER',
 '##T',
 '[MASK]',
 'token',
 '##izer']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [19]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 51264


I0509 07:56:25.364218 140673591916288 tf_logging.py:115] Writing example 0 of 51264


INFO:tensorflow:*** Example ***


I0509 07:56:25.367836 140673591916288 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: None


...

I0509 07:57:16.251547 140673591916288 tf_logging.py:115] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


I0509 07:57:16.252726 140673591916288 tf_logging.py:115] label: 0 (id = 0)


INFO:tensorflow:Writing example 10000 of 12816


I0509 07:57:26.081984 140673591916288 tf_logging.py:115] Writing example 10000 of 12816


#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [20]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [21]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [22]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [23]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS) * 2
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

print(num_train_steps)

9612


In [24]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [25]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_master': '', '_task_type': 'worker', '_tf_random_seed': None, '_experimental_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff0856d6668>, '_save_checkpoints_steps': 500, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_protocol': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_device_fn': None, '_service': None, '_save_summary_steps': 100, '_num_worker_replicas': 1, '_evaluation_master': '', '_is_chief': True, '_task_id': 0, '_model_dir': 'bert_agreement', '_eval_distribute': None, '_train_distribute': None}


I0509 07:57:28.985426 140673591916288 tf_logging.py:115] Using config: {'_master': '', '_task_type': 'worker', '_tf_random_seed': None, '_experimental_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff0856d6668>, '_save_checkpoints_steps': 500, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_protocol': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_device_fn': None, '_service': None, '_save_summary_steps': 100, '_num_worker_replicas': 1, '_evaluation_master': '', '_is_chief': True, '_task_id': 0, '_model_dir': 'bert_agreement', '_eval_distribute': None, '_train_distribute': None}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [26]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [27]:
print('Beginning Training!')
current_time = datetime.now()

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    with tf.device('/gpu:0'):
        estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0509 07:57:52.855791 140673591916288 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0509 07:57:56.336128 140673591916288 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/ivantorubarov/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 9612 into bert_agreement/model.ckpt.


I0509 09:00:08.202982 140673591916288 tf_logging.py:115] Saving checkpoints for 9612 into bert_agreement/model.ckpt.


INFO:tensorflow:Loss for final step: 0.00011041749.


I0509 09:00:11.737573 140673591916288 tf_logging.py:115] Loss for final step: 0.00011041749.


Training took time  1:02:42.696746


Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

Now let's use our test data to see how well our model did:

In [28]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [34]:
estimator.evaluate(input_fn=test_input_fn, checkpoint_path="./bert_agreement/model.ckpt-8000", steps=None)

INFO:tensorflow:Calling model_fn.


I0509 09:08:52.314226 140673591916288 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0509 09:08:55.711827 140673591916288 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/ivantorubarov/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 8000: auc = 0.8188098, eval_accuracy = 0.9661361, f1_score = 0.70596194, false_negatives = 280.0, false_positives = 154.0, global_step = 8000, loss = 0.23432721, precision = 0.77185184, recall = 0.65043694, true_negatives = 11861.0, true_positives = 521.0


I0509 09:09:57.433156 140673591916288 tf_logging.py:115] Saving dict for global step 8000: auc = 0.8188098, eval_accuracy = 0.9661361, f1_score = 0.70596194, false_negatives = 280.0, false_positives = 154.0, global_step = 8000, loss = 0.23432721, precision = 0.77185184, recall = 0.65043694, true_negatives = 11861.0, true_positives = 521.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 8000: ./bert_agreement/model.ckpt-8000


I0509 09:09:57.435011 140673591916288 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 8000: ./bert_agreement/model.ckpt-8000


{'auc': 0.8188098,
 'eval_accuracy': 0.9661361,
 'f1_score': 0.70596194,
 'false_negatives': 280.0,
 'false_positives': 154.0,
 'global_step': 8000,
 'loss': 0.23432721,
 'precision': 0.77185184,
 'recall': 0.65043694,
 'true_negatives': 11861.0,
 'true_positives': 521.0}

# Extracting the trained model


Now let's save our model:

In [47]:
def getPrediction(in_sentences):
  labels = ["Not an error", "Is an error"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x[0], text_b = x[1], label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn, checkpoint_path="./bert_agreement/model.ckpt-8000")
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [37]:
import nltk
import re
import itertools

nltk.download('perluniprops')
nltk.download('punkt')

[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/ivantorubarov/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/ivantorubarov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [38]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from nltk.tokenize.moses import MosesDetokenizer
detokenizer = MosesDetokenizer()

def annotate(text):
  current_time = datetime.now()
  raw = text
  raw = re.sub(r'\s', ' ', raw)
  raw = re.sub(r'( )+', ' ', raw)
  sentences = nltk.sent_tokenize(raw)
  wordsoup = []
  annotation_set = []
  checking_ids = []
  tokens = []
  J = 0
  for sentence in sentences:
    wordsoup.append(tknzr.tokenize(sentence))
  for i in range(len(wordsoup)):
    tokenized_sentence = wordsoup[i]
    for j in range(len(tokenized_sentence)):
      token = tokenized_sentence[j]
      if not re.search(r'[a-zA-Z]', token):
        tokens.append([token, 0])
      else:
        tokens.append([token, None])
        substring = token
        si = i-2
        if si < 0:
          si = 0
        detokenizing_string = wordsoup[si:i] + [tokenized_sentence[:j] + ['[MASK]'] + tokenized_sentence[j+1:]] + wordsoup[i+1:i+3]
        detokenizing_string = [item for sublist in detokenizing_string for item in sublist]
        entry = detokenizer.detokenize(detokenizing_string, return_str=True)
        annotation_set.append([entry, substring])
        checking_ids.append(J)
      J += 1
  predictions = getPrediction(annotation_set[:128])
  for p in range(len(predictions)):
    if predictions[p][-1] == 'Is an error':
      tokens[checking_ids[p]][1] = 1
    else:
      tokens[checking_ids[p]][1] = 0
  print('\n')
  print("Annotation took time ", datetime.now() - current_time)
  return tokens

def print_annotated_webpage(tokens, title):
  out = '<html>\n<head>\n<title>'+title+'</title>\n<meta charset="utf-8">\n<style type="text/css">\n.blue {\n\tbackground: #a8d1ff;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>'
  outsoup = []
  for token in tokens:
    if token[1] == 1:
      token[0] = '<div class="blue">'+token[0]+'</div>'
    outsoup.append(token[0])
  bodystring = detokenizer.detokenize(outsoup, return_str=True)
  out += bodystring
  out += '</body>\n</html>'
  return out

In [48]:
import json

curname = "wsites_" + modelname + ".json"

test_texts = {"./data/old IELTS/IELTS2016/OR_129_2.txt": "Nowadays more and more children are lacking support and attention from their parents. This newfound trend is becoming even more severe with the flow of time. One of the reasons of this situation occurring are unachieved career goals of parents. Financial wellfare and high place on the career stairs are one of the traits of successfull man today. Therefore sponses, especially younger ones, who hasn\u2019t achieved their career goals yet, are said to spend more time working, rather than with children. Another popular explanation of spending less time with kids is the desire of parents of parents to provide their child with best accomodations, toys, clothes and food. This is done by taking more job tasks or by taking another, part-time jobs. It is a well-known fact, that children require upbringing of their parents, and this discussed situation can cause several problems for your child. The first, and an obvious one, are bad moral traits of kids. When there is noone to show what is right and what is knot, child is destined to grow with unmorall views on life. Furthermore there is no barriers to prevent the child from tabacco, alcohol or even drug addictions. Another point is, while it is believed that toys are the sign of love, they can harm your child too. If your baby has more and better toys, than his friends, it is a way of increasing selfishness and proudness into him. To add to this statement, enormous time spent playing toys is proved to have a bad influence on kids brains, limiting their abilities. \n", "./data/exam/exam2017_4/DPe_38_2.txt": "Nowdays there are a lot of problems with internet pirates who steal music and films which leads to a huge losses for producers. In my opinion, such question is very dabatable. That\u2019s why my essay will discuss, firstly, why pirates should be punished, and, secondly, reasons to copy and share music or films on the internet.\nIt\u2019s agreed that copying and sharing original media content is illegal. There are several laws in Russia that try to control these internet segment. For example, there is special government organisathion that blocks resourses with illegal materials. In that case, it is not a secret that it is easier to buy a film in the official store, becouse other way you lose several advantogies. With this in mind , let\u2019s think about the quality of this content and the moral norms. As for my family, we prefer to see a film in the good resolution and we don\u2019t want to break the rulers when we decide to watch a new movie or listen new album.\nOn the other hand, there are some reasones to download music of movies from the unofficial store. The main point is that we should take into accont that we live in Russia and sometimes it closes the oppotunities to find an alboum or a movie from very small foreign group or with unpopular actors. For instance, It is too difficult to find The Big Bang theory in Russian stores and also in our language. In that case, we don\u2019t have another oppotunities and have to dounload and translate it from unofficial resources. Moreover, these Russian laws sometimes play the role of instrument for small media producers to earn money from illegal stores or, even more often, from fan cites.\nTo sum up, this question have different sides, but, In my opinion, there should be no illegal media at all.\n", "./data/2012-2014/esl_00225.txt": "The sport is an entertainment that came to us from the ancient Greeks. Since those times every sport event has a competitive nature. However, now money has reached the sport too. An enormous amount of money has been invested in popular sports for many reasons: health care, salaries to sportstars, participation in advertisment or bribing judges. Under such circumstances, can we speak that the sport has kept its competitive nature or the sport is already corrupted by money?\nThe thing is that now everything depends on money, even sport. Money is an ubiquitous totalizator. In sport it helps to support sportsmen in different spheres. Therefore sport is advertised by money. On billboards, on TV, on the Internet there are adverts containing sportstars. It even does not matter what the product is - Coca-Cola, trainers or perfume. Furthermore, money means cheating of any kind. With the help of money sportsmen can get steroids to increase their physical skills or bribe a referee.\nOn the other hand, the true nature of the sport is still the same. Each sportsman struggles to reach the top. Only persistent training and desire to reach the goal lead to the top of the Sports Hall of the Fame. While such a glory does not get easily, a sportsman who surrenders to the training completely can achieve a lot. The life of the sportsmen seems to be full of modern romanticism: tours, competitions, championships, meetings, new friends and experience, cups, medals. However, it is the daily work on the brink of physical abilities. Money is expected by many sportsmen, but only reaching their goals, the sportsman feels satisfaction and peace.\nSummarising the above, I would like to notice that the more money the sport gains the more it seems to move away from its core values. However, I partly disagree. The whole idea of the sport is compatition. It will not be moved away only because of money. The sport exists until its compatitive nature exists.", "./data/old IELTS/IELTS2016/EKu_6_1.txt": "Both line graph and bar chart provide information about the money spent on renewable energy in the world over the period of 7 years. They shows the features both od developed and developing countries and clearly shows that they are not equal at all. Overall, it is easy to see from the line graph that the quantity of money spent on this kind of energy by a world total takes its minimum at just 100 $ Bn and reaches its peak at the feature equal to 279 $ Bn constantly\u00a0increasing from 2006 to 2011. Then by 2013 it becomes slightly lower. According to the bar graph, the features of developed countries are significantly higher then ones of developing countries (so the highest rates for developed and developing countries are approximately 180 $ Bn and 105 $ Bn respectively. However, it can be seen that while the rates of developed countries are increasing almost the all shown period (from 25 $ Bn approximately to , there is no such tendence for the rates of developed ones. This is the end of my report.", "./data/exam/exam2017/EGe_190_2.txt": "There is no doubt, that nowdays there are plenty of British and American films in the cinemas. What is more, it becomes more and more possible to watch the original films in english with or without subtitles.\nIn my opinion, the popularity of British and American films can be explained with economical factors. As the american and british economy are the first world economies through the last decades (or even a century), there was an opotunity to support financially local industries. Therefore, there are a lot of money spent on films and there are a lot of people, who work in the british and american film industries, which turns out to be an effective way to develop film making.\nFrom the other side, one of the reasons for that succes is that the british and american culture has a great influence all over the world. English is a widely spoken language, it is the international language. That is why it is much more possible to spare the english and american films, to translate them. As we all know, one of the most enjoyable way to learn language of some country is to participate in its culture, to watch films, to listen to music.\nThat is why, in my opinion, if governments have as a goal the development of local film industries, they should spend more money on local films and look carefully that they are spent for the good reasons\nTo conclude, I would say that we should inspire people in our country to create better films, we should show our interest in quality of the product.", "./data/old IELTS/IELTS2016/JSl_41_2.txt": "There are widespread opinion about repercussions of air travelling like contribution of air pollution and global warming. Some people think that the good solution of this problems is to decline the number of unnecessary travelling by air. I have to agree with those who think that the government should protect the unnecessary travel by laws, because planes damage our environment too seriously. First of all government can reduce the amount of travel in aim of work. For instance, businessmans can solve some problems with help of technique and online services. They can just plane their business partner or call them in the Skype, for example. It can be difficulties with identifications of persons who can flying for a business questions and who can not, but it can be solved by changing the process of buying tickets for air travel with entering the stage of writing the aim of travel in colours. What about the traveling for leisure like summer holidays travel, there are can not be restrictions in the peack-time when a lot of people have their holidays and travel. However, it seems me that in other periods, when the most of citizens is working, governments can raise the taxes on air travel which depends on tickets costs and it can reduce the quantity of air travel with following declining the air pollution of planes. To conclude, government should to protect the unnecessary air travels by laws, because it is one of the causes of global warming and environment solution. For the first time people can negatively comment such restictions but it is the piece of cake for person to limit his or her air travelling. If people wants to save the environment for their children.  \n", "./data/exam/exam2014/MGr_13_2.txt": "Many people want to be happy, but happiness means different for different people. To my mind, the definition of happiness depends on people, because different factors determine every personality. However, it is very important for each person to feel happy. How we can define, what is happiness and what do we need to do to achieve it?\nI agree with statement that although the happiness is very important in life, it is very difficult to define what is it. To begin with, our personalities are determined by some factors like the level of life, habitats, education, family, culture. These factors form our needs and wants. For each person happiness means different because the features of culture, different worldview, goals. For example, people can feel happy if they have enough food every day, but for people from modern cities with the high pece of life it is not enough. On the contrary, they have some goals, which are connected with career and money. In this case happiness can mean success in different spheres of life.\nIt is generally accepted that a lot of philosophers tries to define what happiness is. However, they have not an answer for this question. It is very clear from these observations that it was always difficult to determine what makes people happy. Moreover, there are a lot of books about love and success and strong people, who can cope with difficulties. But, essentially, it is argued that this issue is discussion for humankind.\nWe dream about being happy. Nevertheless, people often do was know how to achieve happiness. I think that it is impossible to say whos factors are important in achieving happiness because each person wants feel happy in his personal way. According to some experts, the exteut of happiness in different countries depends on the level of life and opportunities to realize themselves. As for me, the main factor for achieving happiness is having family and friends, who love and support you.\nIn conclusion, I would like to say that happiness is very difficult to define because people have different willings and needs, which depends on different factors. What is more there are no particular factors to achieving happiness. If you want to be happy, just be happy without some thoughts about money or your willings.\n", "./data/old IELTS/IELTS2015/EPa_31_1.txt": "The presented charts are giving us the statistical information about the population of Yemen and Italy, dividing it on sectors by the age. The information given is describing the situation current for the year of 2000 and predicting it to 2050. \nThe first feature we should pay attention on is the gap between the amount of kids younger than 14 in these countries. Besides the possibility that this number will become smaller in both countries in next fifty years, the gap will remain but also diminue.\nAlso, both countries will probably rise their average maximal age, as it can be seen in the prediction on charts. The amount of old people will grow almost two times by 2050th. \nThose tho thrends can change the economical situation in both states. But if Yemen labour market will grow because of increasing of people fifteen to sixty years old, the situation in Italy is completely opposite because the amount of retired people is already significantly bigger and tend to grow.\n", "./data/exam/exam2017_6/OBy_112_1.txt": "The bar chart illustrates the information about unemployment rate in selected regions and compare two years information.\n\nAs is observed from the illustration, the percentage of N.Africa was simmelar in 2014 and 2015 years. The same situation about worldwide. And in S.Asia.\n\nSecondly, in the Middle East and Latin America level of unemployment was higher in 2014 then 2015. In the Middle East it was 11% in 2014 then get lower to 9,6%. In Latin America higher point was in 2014, it\u2019s 6,8%, then decreased to 5,8%.\n\nThe different between others colomns of the graph was in EU. Wich rate was get a high in 2015, with 7,9% and become lower in 2014, it is now 7,5%.\n\nSummarising the all information can bee seen that N.Africa, S.Asia and worldwide regions have the simmelar unemployment rate percents in both years. The Latin America reach a pic in 2014 and EU in 2015.\n", "./data/exam/exam2014/ZEv_35_2.txt": "Nowadays, education quality has been improved noticeably, serious problems with students behaviours are still happening. Let\u2019s observe the issue.\n\nTo begin with, I would like to say that one reason is the education from student\u2019s families, their parents. Parents don\u2019t have a lot of free time, because they only work. To reach a fortune is value for parents nowadays. Also, two reason is a bad education at school. I believe, that a lots students dislike schools, because subjects don\u2019t interesting and typical. Also, pupils don\u2019t have pleasure of subjects, that also change of mood them and countries schools have severe problems with student behaviour.\n\nTo continue the topic, I can say, that this problem with bad behaviour of pupils can solve. In my opinion, parents must spend make time with their children. Certainly, they must go for a walk together, visit some museums and communicate with daughter and son. As for me, my family spend time with me, talk with me. Also, more important, schools should know students, who are not well-behaved so as to reeducate them, preventing them from affecting other students.\n\nIn conclusion, it is not only responsibility of families and schools but also the whole society to solve behavioral problems. In addition, those solutions mentioned above should be taken action before the problems becoming too bad to be fixed. \n", "./data/2012-2014/esl_00437.txt": "6. Is lying ever permissible?\nDo you really want to be a person for whom the lie is giving regularly?\nI think that the majority of the people will answer that they prefer to know truth. Nevertheless the part of people expresses the ability to except lie and accept  it.  The reality is that we are living in the society where we can see the lie since the birth till death. Our surrounding permits to lie and does not block it on the first stages. The person adherences the live situations and allows him/herself to act according to it.\nWhy the child begins to lie to his parents? I was the observer of the situation when the young brother of my friend came to his parents and said that he had broken his bicycle. The parents reacted emotionally and made the young boy guilty in situation which had happened. In the reality the boy was not guilty. The bicycle has been broken due to an accident which had had occurred in the yard. The parents did not understand the issue completely. They possible provoked the feeling that truth has not been found out and has not been accepted. The second time there would not be any reason for young boy to describe the complete situation.\nAnalyzing this point, I can only emphasize that sometimes we ourselves gives the prepositions for lie.\nFrom another point, by our nature we would like to know truth. We would prefer to understand the real positions of the person and his views even if we do not accept it. We need the truth for improvement and development. I would prefer to know if my cookies are tasty or not. Imagine, you say that cookies are tasty, but in reality you could not eat even one. On the other hand, you say the truth and help the person to develop skills of cooking. \nI have developed the understanding that it is the choice of each person to tell or not to tell truth or lie. The live gives us the experience based on which the conclusion is made .Even thought the society admits the lie and thus the lie is permissible.\n", "./data/exam/exam2016/OR_48_2.txt": "There if the problem with air travels because they impact on environment and improve global warming. As a result, the government wants to delay travels for business and leisure. As for me, I agree with a big importance of climat and air pollution but I think that this law will make more problems. I have three explanations why the law which reduces amount of business and leisure travels is a bad idea. To start with, the business travel is very important for a communication between companies from different countries. This communication provides countries by new financial assets which help for economy growth. If the government prohibit business travel, the stream of money can confront a lot of difficults and foreign companies will stop communicating. Then the country will threat a lot of losses which slowen the economic growth. The second point is a possible lack of entertainment for population. If they like relaxing at other countries, the government shouldn\u2019t prohibit to do it. Because there are countries which specialized on leasure and it is their part of the world work distribution. The last but not least, if government doesn\u2019t allow to travel to other countries, they should make own entertainment sphere which replaces foreign one. However, it is a big investment and a lot of time which are not available for some countries. The opposite side supposes that the air pollution is a very crucial problem and each country should fights against if because of global warming. It is dangerous because of a raising temperiature which kills some types of animals and plants. However, plants produce clear air for people in order to the global warming is a serious problem for future generations. As for the second point of view, I believe that there are a lot of other problems which causes global warming such as waste, car traffic and others. So that I suggest that the government should reduce impact of these factors which help people and environment. All in all, the law reducted amount of air travel is illogical and global warming is caused by a lot of factors which are easy for interacting.\n", "./data/exam/exam2014/EPa_7_2.txt": "Nowadays, more and more people unhealthy and who have different problem with health. I agree with this statement. I think that created new the number of sports facilities. Is good idea. But also I agree with people who say that it is little effect to do people health. In modern life people don`t can without different technology such as computer, mobile phone, car and other. People very busy in 21 centery and don`t have a time on sports exercises. Instead they begining smoke, drink alchohol. Modern population needed in advertising good health. People have to understand that thae needed in sports. Public health very important for society. Different competitions helps people to achieve success in different sphere. Also people can communicated with other people and knows new information. Increase the number of sports facilities. On my opinion have to help people think that they  improve public health.\nAlso in modern society have people who cared about health. They visited fitness clubs to do different sports exersice. Also they eat good and healthy product such as vegetables, meat which cooked on water, (hot air) and drink juice, also they drink different vitamins.\nIn conclusion I want to say that public health is very important for people. Because it is very, very important for two next population, how much people live in world in the features and how they will develop in the features. Created new sports  facilities is good idea because people look that they can visited different sports and to go in sports.\n", "./data/exam/exam2016/EKu_25_1.txt": "The information given demostrates changes in investment in renewable energy all over the world from 2006 to 2013 in developed and developing countries, as well as a world total. The data are provided in $ Bn.\u00a0 It can be deduced from the graph that there were a stable growth in investment from 2006 to 2009 (100 $ Bn and 168 $ Bn relatively), and investment in developing countries was growing steadily, while the figures in developed countries remained stable. Later, the amount of investment increased dramatically to 227 $ Bn in 2010 and 279 $ Bn in 2011, mostly because of significant growth in investment in developed countries.\u00a0 Overall, after analysing the information given, it can be stated that the world total in the investment in renewable energy increased at more than 2 times from 100 $ Bn in 2006 \u00a0to 214 $ Bn in 2013: there was a steady growth until 2011, then a world total started decreasing roughly at the same pace.\u00a0", "./data/exam/exam2017_2/ABl_27_2.txt": "In fact pirate copies led to different producers lose their benefits every year. That is why a lot of people think that downloading of music or films without paying money for that production should be punished. There are some advantages and disadvantages of this point of view.\nFirst and foremost, people spend a lot of money, huge sums to do a new film or music composition and it is unfaire when they are not able to get their money. Internet survesis get access to cope and share of somebody\u2019s production. Perhaps, this types of sites should be block. And then authors\u2019 rights will not break.\nMoreover, arctors, producers, musicians, directors and other people who work in this industry should get their salary for their job. Sometimes people do understand that produsing of films is very expensive. Legal copies let to get a compensation.\nThat is why people who find and download files just stil them. And it is a type of crime. People who do crime should be punished. Different punishment led to increase of buying the original product (files, discs).\nHowever there is another point of view. Persons who download pirate copies are not muders. They just try to share with people moves and beautiful music compusitions. Of course, some people feel like spending money for buying the legal copies is not productive and it can be too costy spend money to every film or disc of music. They do not care about people who make this production. In their oppinion, copying and sharing music or films on the Internet help them save money.\nTo sum up, punishmend for pirate copies should be in the state to provide authors\u2019 rights, but in my oppinion they should not be very strict. Maybe shtrafs will be very suitable. can help producers get their benefits and will be useful for goverment and budget.\n"}
outie = [print_annotated_webpage(annotate(test_texts[tt]), tt) for tt in test_texts]
with open(curname, 'w', encoding="utf-8") as w:
  json.dump(outie, w)

INFO:tensorflow:Writing example 0 of 128


I0509 09:29:38.594300 140673591916288 tf_logging.py:115] Writing example 0 of 128


INFO:tensorflow:*** Example ***


I0509 09:29:38.597122 140673591916288 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0509 09:31:25.007574 140673591916288 tf_logging.py:115] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0509 09:31:25.088447 140673591916288 tf_logging.py:115] Done running local_init_op.




Annotation took time  0:00:07.732230


In [40]:
os.listdir(".")

['.ipython',
 '.cache',
 '.ipynb_checkpoints',
 'bert_prepositions',
 'delete.json',
 '.jupyter',
 '.bashrc-anaconda3.bak',
 '.config',
 'REALEC_AutoAnnotator_BERT_pipeline Often_confused.ipynb',
 'Prepositions_test.json',
 'wsites_OftenConfused_ToFifteen.json',
 'Often_confused_test.json',
 'Often_confused.json',
 'anaconda3',
 'Anaconda3-4.0.0-Linux-x86_64.sh',
 'Agreement_errors_test.json',
 'download_google_drive',
 '.nano',
 '.bash_history',
 'wsites_Prepositions_ToFifteen.json',
 'Capitalisation.json',
 'Prepositions.json',
 'gdrive-linux-x64',
 '.keras',
 'delete_test.json',
 '.python_history',
 'nltk_data',
 'wsites_delete_ToFifteen.json',
 'wsites_Capitalisation_ToFifteen.json',
 'Datasets',
 '.local',
 'wsites_lex_item_choice_AugmentedRatio.json',
 'REALEC_AutoAnnotator_BERT_pipeline Capitalisation.ipynb',
 'jupyter-drive',
 'cuda-repo-ubuntu1604_9.0.176-1_amd64.deb',
 '.continuum',
 '.conda',
 '.bashrc',
 'wsites_Agreement_ToFifteen.json',
 'bert_new_prepositions',
 '.nv',
 

In [49]:
outie[4]

'<html>\n<head>\n<title>./data/exam/exam2014/EPa_7_2.txt</title>\n<meta charset="utf-8">\n<style type="text/css">\n.blue {\n\tbackground: #a8d1ff;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>Nowadays, more and more people unhealthy and who <div class="blue">have</div> different <div class="blue">problem</div> with health. I agree with this statement. I think that created new the number of <div class="blue">sports</div> facilities. Is good idea. But also I agree with people who say that it is little effect to do people health. In modern life people don `t can without different technology such as computer, mobile phone, car and other. People very busy in 21 centery and don` t have a time on <div class="blue">sports</div> exercises. Instead they begining smoke, drink alchohol. Modern population needed in advertising good health. People have to understand that thae needed in <div class="blue">sports</div>. Public health very important for society. Different <div class="blue">com

In [50]:
!./gdrive-linux-x64 upload wsites_Agreement_ToFifteen.json

Uploading wsites_Agreement_ToFifteen.json
Uploaded 1knIV-aYgvlEusthHx9BNnS7M2KJUq6Pq at 25.2 KB/s, total 28.4 KB


In [31]:
!ls -lAh ./bert_agreement

total 7.3G
-rw-rw-r-- 1 ivantorubarov ivantorubarov  271 May  9 09:00 checkpoint
drwxr-xr-x 2 ivantorubarov ivantorubarov 4.0K May  9 09:01 eval
-rw-rw-r-- 1 ivantorubarov ivantorubarov 161M May  9 07:37 events.out.tfevents.1557387376.instance-2
-rw-rw-r-- 1 ivantorubarov ivantorubarov 322M May  9 09:00 events.out.tfevents.1557388700.instance-2
-rw-rw-r-- 1 ivantorubarov ivantorubarov 306M May  9 07:58 graph.pbtxt
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G May  9 08:49 model.ckpt-8000.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K May  9 08:49 model.ckpt-8000.index
-rw-rw-r-- 1 ivantorubarov ivantorubarov  81M May  9 08:49 model.ckpt-8000.meta
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G May  9 08:53 model.ckpt-8500.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K May  9 08:53 model.ckpt-8500.index
-rw-rw-r-- 1 ivantorubarov ivantorubarov  81M May  9 08:53 model.ckpt-8500.meta
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G May  9 08:56 model.ckpt-

In [ ]:
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2396.data-00000-of-00001
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2396.index
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2396.meta

Uploading ./bert_output/model.ckpt-2396.data-00000-of-00001
234.9 MB/1.3 GB, Rate: 41.1 MB/s                  

In [1]:
!rm -rf ./bert_output

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': curname})
uploaded.SetContentFile('./'+curname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 16_pmUKOHS5lKXM2o-1QyVcPQohalCRZd


# Preparing existing model for REALEC production